# 第一部份:取得台股日線資料
我們這邊介紹二個方式取得台股的日線資料，一個是到台灣證券交易所抓取資料，我們也可以利用已經寫好的套件庫去抓取。
- [臺灣證券交易所](http://www.twse.com.tw/zh/)
- [twstock套件](https://github.com/mlouielu/twstock)

## 臺灣證券交易所


In [83]:
import requests
import pandas as pd
import io
import re
import time
import datetime

In [80]:
def crawler(date_time):
    page_url = 'http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + date_time +'&type=ALLBUT0999'
    page = requests.get(page_url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
        if text == '"證券代號","證券名稱","成交股數","成交筆數","成交金額","開盤價","最高價","最低價","收盤價","漲跌(+/-)","漲跌價差","最後揭示買價","最後揭示買量","最後揭示賣價","最後揭示賣量","本益比",':
            initial_point = i
            break
    test_df = pd.read_csv(io.StringIO(''.join([text[:-1] + '\n' for text in use_text[initial_point:]])))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('"',''))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x: x.replace('=',''))
    return test_df

In [123]:
#抓好幾天
def trans_date(date_time):
    return ''.join(str(date_time).split(' ')[0].split('-'))

def parse_n_days(start_date,n):
    time.sleep(5)
    df_dict = {}
    now_date = start_date
    for i in range(n):
        now_date = now_date - datetime.timedelta(days=1)
        try:
            df = crawler(trans_date(now_date))
            print("Current date" + trans_date(now_date))
            df_dict.update({trans_date(now_date):df})
            print('Successful!!')
        except:
            print('Fails at' + str(now_date))
    return df_dict

## twstock套件

In [125]:
from twstock import Stock

stock = Stock('2330')

In [126]:
type(stock)

twstock.stock.Stock

In [129]:
stock_price = stock.fetch_31()

# 第二部份:練習用的高頻資料
FXCM的模擬帳戶提供高頻資料的api獲取
- Step1:[開啟免費模擬帳戶](https://www.fxcm.com/uk/forex-trading-demo/) 
- Step2:創造 API Token

In [165]:
import fxcmpy
api = fxcmpy.fxcmpy(access_token=API_key, server='demo')

In [166]:
from fxcmpy import fxcmpy_tick_data_reader as tdr

In [167]:
print(tdr.get_available_symbols())

('AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'CADCHF', 'EURAUD', 'EURCHF', 'EURGBP', 'EURJPY', 'EURUSD', 'GBPCHF', 'GBPJPY', 'GBPNZD', 'GBPUSD', 'GBPCHF', 'GBPJPY', 'GBPNZD', 'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD', 'USDCAD', 'USDCHF', 'USDJPY')


In [170]:
start = datetime.datetime(2018, 2, 1)
end = datetime.datetime(2018, 2, 15)

In [171]:
dr = tdr('EURUSD', start, end, verbosity=True)

Fetching data from: https://tickdata.fxcorporate.com/EURUSD/2018/5.csv.gz
Fetching data from: https://tickdata.fxcorporate.com/EURUSD/2018/6.csv.gz
Fetching data from: https://tickdata.fxcorporate.com/EURUSD/2018/7.csv.gz


In [172]:
dr.get_raw_data().info()

<class 'pandas.core.frame.DataFrame'>
Index: 5643477 entries, 01/28/2018 22:00:46.425 to 02/16/2018 21:59:08.423
Data columns (total 2 columns):
Bid    float64
Ask    float64
dtypes: float64(2)
memory usage: 129.2+ MB


In [174]:
dr.get_data().info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5643477 entries, 2018-01-28 22:00:46.425000 to 2018-02-16 21:59:08.423000
Data columns (total 2 columns):
Bid    float64
Ask    float64
dtypes: float64(2)
memory usage: 129.2 MB


In [175]:
dr.get_data().head()

,Bid,Ask
2018-01-28 22:00:46.425,1.24239,1.24282
2018-01-28 22:00:46.438,1.24236,1.24282
2018-01-28 22:00:59.117,1.24236,1.24267
2018-01-28 22:00:59.134,1.24229,1.24267
2018-01-28 22:01:25.668,1.24228,1.24266
